In [1]:
import pandas as pd

df = pd.read_csv('corpus.csv')
df

,Unnamed: 0,artist,title,year,album,lyrics,n_words,pageviews,contributors,url
0,1,Dry,94310,2012.0,Tôt ou tard,"[Paroles de ""94310"" ft. Madj, Larso & Stokas]\...",748,0,8,https://genius.com/Dry-94310-lyrics
1,2,Mafia K’1 Fry,Au bon vieux temps,2007.0,Jusqu’à la mort,"\nEt je les entends tous les Zoulous, ils parl...",958,7514,8,https://genius.com/Mafia-k1-fry-au-bon-vieux-t...
2,3,DJ Hamida,Attrape-Moi Si Tu Peux,NaN,Mix Party 2015,[Refrain : Charly Bell]\nOui je sais qu’t’as e...,367,0,4,https://genius.com/Dj-hamida-attrape-moi-si-tu...
3,4,Kery James,94 c’est le Barça Remix,NaN,NaN,"\nC'est pour les rudeboys, c'est pour les cail...",535,0,7,https://genius.com/Kery-james-94-cest-le-barca...
4,5,Dry,14 ans déjà,2013.0,Maintenant ou jamais,"[Paroles de ""14 ans déjà""]\n\n\nTu sais, j'cro...",653,5037,8,https://genius.com/Dry-14-ans-deja-lyrics
...,...,...,...,...,...,...,...,...,...,...
85018,85019,Bricksy & 3g,Akai,2024.0,But It Ain’t,"[Paroles de ""Akai"" ft. La Fève]\n\n\nFF7\nWalo...",471,0,7,https://genius.com/Bricksy-and-3g-akai-lyrics
85019,85020,Bricksy & 3g,I am GIA,2024.0,But It Ain’t,"[Paroles de ""I am GIA""]\n\n\nSa star\nIl me di...",522,0,4,https://genius.com/Bricksy-and-3g-i-am-gia-lyrics
85020,85021,Rafal,ROMEO,2024.0,NaN,\n\n\nPrépare le sparadrap\nJ'péterais ma tête...,393,0,4,https://genius.com/Rafal-romeo-lyrics
85021,85022,Aktéfräzé,"Alpha, kilo, tango, [é]cho... Triple comme au ...",1999.0,B.O.S.S. Vol. 1,\nComme au scrabble et...\n\n\nLa fine fleur d...,464,0,1,https://genius.com/Aktefraze-alpha-kilo-tango-...


# Sexism analysis

Huggingface model :  https://huggingface.co/annahaz/xlm-roberta-base-misogyny-sexism-indomain-mix-bal

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("annahaz/xlm-roberta-base-misogyny-sexism-indomain-mix-bal")
model = AutoModelForSequenceClassification.from_pretrained("annahaz/xlm-roberta-base-misogyny-sexism-indomain-mix-bal")

c:\miniconda\envs\env_1\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
# text = 'Ceci est une phrase neutre'
# text = 'Bonjour, je vous trouve très jolie'
text = 'T\'es bonne ouech'
# text = 'Chez moi, que de la bonne substance, normal elle sort d\'Amsterdam'
# text = 'C\'est une salope'
# text = 'Je vais te la mettre si t\'ouvres tes fesses'


inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

print(logits)

predicted_class_id = logits.argmax().item()
# model.config.id2label[predicted_class_id]
predicted_class_id

tensor([[ 2.7519, -3.4935]])


0

In [4]:
import re

def clean_text(text):
    # text = text.replace('\n', ' ')
    # text = text.replace('\r', ' ')
    text = re.sub(r'\[.*?\]', '', text) # remove text between brackets (title)
    text = re.sub(r'[^\w\s]', ' ', text) # remove punctuation
    return text

def violence_probability(line):
    inputs = tokenizer(line, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    # predicted_class_id = logits.argmax().item()
    # return model.config.id2label[predicted_class_id]
    return logits

def categorize(lyrics):
    # for each line of the lyrics, get the probability of violence and return the mean
    lyrics = clean_text(lyrics)
    lines = lyrics.split('\n')
    lines = [line for line in lines if line] # remove empty lines
    # print(lines)
    probabilities = [violence_probability(line) for line in lines]
    # print(probabilities)

    mean_probability = sum(probabilities) / len(probabilities)
    # print('mean probability : ', mean_probability)    
    predicted_class_id = mean_probability.argmax().item()
    mean_proba_classe = model.config.id2label[predicted_class_id]

    # get the class of each line
    classes = [model.config.id2label[proba.argmax().item()] for proba in probabilities]
    # print(classes)
    nb_violent_lines = len([cl for cl in classes if cl == '1'])
    nb_non_violent_lines = len([cl for cl in classes if cl == '0'])
    ratio_violent_lines = nb_violent_lines / (nb_violent_lines + nb_non_violent_lines)
    # print('ratio violent lines : ', ratio_violent_lines)

    return mean_probability, ratio_violent_lines

lyrics = df['lyrics'][0]
categorize(lyrics)

(tensor([[ 1.6209, -2.1076]]), 0.21875)

In [6]:
df_alkpote = df[df['artist'] == 'Alkpote']
for i in range(10):
    lyrics = df_alkpote.iloc[i]['lyrics']
    # print('lyrics : ', lyrics)
    print(categorize(lyrics))
    # print('\n\n\n\n')

(tensor([[-0.1422, -0.0194]]), 0.5888888888888889)
(tensor([[ 0.8407, -1.2274]]), 0.43283582089552236)
(tensor([[ 1.1938, -1.5892]]), 0.25)
(tensor([[ 1.5284, -1.9794]]), 0.24528301886792453)
(tensor([[ 1.2478, -1.6807]]), 0.28735632183908044)
(tensor([[ 1.5624, -2.0617]]), 0.18556701030927836)
(tensor([[ 0.5954, -0.9085]]), 0.4074074074074074)
(tensor([[ 1.1115, -1.5045]]), 0.2807017543859649)
(tensor([[-0.6203,  0.5877]]), 0.6111111111111112)
(tensor([[ 1.7283, -2.2697]]), 0.16666666666666666)


In [77]:
df.columns

Index(['Unnamed: 0', 'artist', 'title', 'year', 'album', 'lyrics', 'n_words',
       'pageviews', 'contributors', 'url', 'clean_lyrics', 'tokens'],
      dtype='object')

In [ ]:
most_popular_artists = df[['artist', 'pageviews']].groupby('artist').sum().sort_values(by='pageviews', ascending=False).head(10)
df[df['artist'].isin(most_popular_artists.index)]

In [7]:
# get 10 most popular artists (by number of pageviews on total on genius)

most_popular_artists = df[['artist', 'pageviews']].groupby('artist').sum().sort_values(by='pageviews', ascending=False).head(10)

# apply categorize function to each artist
for artist in most_popular_artists.index:
    print('artist : ', artist)
    df_artist = df[df['artist'] == artist]
    for i in range(len(df_artist)):
        print('song : ', i)
        lyrics = df_artist.iloc[i]['lyrics']
        # print('lyrics : ', lyrics)
        print(categorize(lyrics))
        # print('\n\n\n\n')


artist :  Booba
song :  0
(tensor([[ 1.7870, -2.3188]]), 0.21212121212121213)
song :  1
(tensor([[ 1.4541, -1.9047]]), 0.23076923076923078)
song :  2
(tensor([[ 1.1447, -1.5200]]), 0.2926829268292683)
song :  3
(tensor([[ 1.2992, -1.7314]]), 0.225)
song :  4
(tensor([[ 1.6250, -2.1585]]), 0.2)
song :  5
(tensor([[ 2.4419, -3.0426]]), 0.07894736842105263)
song :  6
(tensor([[ 2.0552, -2.6371]]), 0.13333333333333333)
song :  7
(tensor([[ 3.4965, -4.1965]]), 0.0)
song :  8
(tensor([[ 0.9775, -1.3180]]), 0.358974358974359)
song :  9
(tensor([[ 1.8567, -2.4071]]), 0.17647058823529413)
song :  10
(tensor([[ 0.4215, -0.6822]]), 0.41025641025641024)
song :  11
(tensor([[ 1.6453, -2.1467]]), 0.2222222222222222)
song :  12
(tensor([[ 0.4174, -0.6488]]), 0.4166666666666667)
song :  13
(tensor([[ 1.1362, -1.5420]]), 0.3048780487804878)
song :  14
(tensor([[ 0.8337, -1.1618]]), 0.3472222222222222)
song :  15
(tensor([[ 1.3778, -1.7671]]), 0.26785714285714285)
song :  16
(tensor([[ 2.0304, -2.5669]]

ZeroDivisionError: division by zero

# Tokenization

In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\alexi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [11]:
from nltk.tokenize import word_tokenize

sample_text = "Une phrase en français et avec un accent étizofnoz. Une balle dans la tête"
tokens = word_tokenize(sample_text)
tokens

['Une',
 'phrase',
 'en',
 'français',
 'et',
 'avec',
 'un',
 'accent',
 'étizofnoz',
 '.',
 'Une',
 'balle',
 'dans',
 'la',
 'tête']

In [ ]:
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# text_to_tokenize = 'This is a sample text'
# tokens = tokenizer.tokenize(text_to_tokenize)
# tokens

# df['tokens'] = df['lyrics'].apply(tokenizer.tokenize)
# df['tokens']

['this', 'is', 'a', 'sample', 'text']

In [ ]:
import re

def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.replace('\r', ' ')
    text = re.sub(r'\[.*?\]', '', text)
    # remove punctuation and replace by space
    text = re.sub(r'[^\w\s]', ' ', text)
    return text

0        [Paroles de "94310" ft. Madj, Larso & Stokas]\...
1        \nEt je les entends tous les Zoulous, ils parl...
2        [Refrain : Charly Bell]\nOui je sais qu’t’as e...
3        \nC'est pour les rudeboys, c'est pour les cail...
4        [Paroles de "14 ans déjà"]\n\n\nTu sais, j'cro...
                               ...                        
85018    [Paroles de "Akai" ft. La Fève]\n\n\nFF7\nWalo...
85019    [Paroles de "I am GIA"]\n\n\nSa star\nIl me di...
85020    \n\n\nPrépare le sparadrap\nJ'péterais ma tête...
85021    \nComme au scrabble et...\n\n\nLa fine fleur d...
85022    Un début de janvier, si j'ai bien su compter\n...
Name: lyrics, Length: 85023, dtype: object

In [28]:
word_tokenize(clean_text(str(df['lyrics'][0])))

df['tokens'] = df['lyrics'].apply(str).apply(clean_text).apply(word_tokenize)
df['tokens']

0        [Un, balle, dans, la, tête, je, te, la, place,...
1        [Et, je, les, entends, tous, les, Zoulous, ils...
2        [Oui, je, sais, qu, t, as, envie, On, cherche,...
3        [C, est, pour, les, rudeboys, c, est, pour, le...
4        [Tu, sais, j, crois, qu, j, ai, pas, réalisé, ...
                               ...                        
85018    [FF7, Walone, J, voulais, juste, baiser, le, g...
85019    [Sa, star, Il, me, dit, qu, j, suis, sa, star,...
85020    [Prépare, le, sparadrap, J, péterais, ma, tête...
85021    [Comme, au, scrabble, et, La, fine, fleur, des...
85022    [Un, début, de, janvier, si, j, ai, bien, su, ...
Name: tokens, Length: 85023, dtype: object

In [17]:
text = df['lyrics'][0]
print(text)
text = clean_text(text)
print('clean text\n\n\n\n', text)

tokens = word_tokenize(text)
print(tokens)


[Paroles de "94310" ft. Madj, Larso & Stokas]


Un balle dans la tête, je te la place comme Beckham
Je vais te la mettre si t'ouvres tes fesses, Orly-mite déboule en bécane
Mon son dans l'illicite comme tous ces dealers de came
Le 9.4.310 puissance de frappe, qu'il y ait pas d'amalgame
C'est qui qui fait du bruit, dites-moi c'est quoi ce vacarme ?
C'est S.T.O.K.A.S le mec de tess que personne ricane
(La Rue) pardonne pas donc forcément j'ai pas d'état d'âme
J'suis dans la street, j'suis dans les blocs et derrière moi j'laisse deux trois cadavres
Chez moi, que de la bonne substance, normal elle sort d'Amsterdam
Orly un phénomène, les vrais le savent on vient vous faire mal
J'ai le flair, je le ressens les khey comme n'importe quel animal
J'ai les crocs, j'ai envie de tout niquer les khey donc j'resterai radical
Le Val de Marne (Le Val de Meurtre) restera dans les annales
On défonce au MIC bien pire encore qu'un bon joint de zamal
C'est le 9.4.310, la zone est bouillante, la zone est dan